<a href="https://colab.research.google.com/github/tara618/symmetrical-octo-palm-tree/blob/main/ML_Lab_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('Book1.csv')
df

,age,income,student,credit rating,buys_computer
0,<=30,high,no,fair,no
1,<=30,high,no,excellent,no
2,31 . . .40,high,no,fair,yes
3,>40,medium,no,fair,yes
4,>40,low,yes,fair,yes
5,>40,low,yes,excellent,no
6,31 . . .40,low,yes,excellent,yes
7,<=30,medium,no,fair,no
8,<=30,low,yes,fair,yes
9,>40,medium,yes,fair,yes


In [3]:
buys_computer_column = [row[-1] for row in data]

total_instances = len(buys_computer_column)
buys_computer_no = buys_computer_column.count('no')
buys_computer_yes = buys_computer_column.count('yes')

prior_no = buys_computer_no / total_instances
prior_yes = buys_computer_yes / total_instances

print(f'Prior probability for "buys_computer = no": {prior_no:.3f}')
print(f'Prior probability for "buys_computer = yes": {prior_yes:.3f}')

Prior probability for "buys_computer = no": 0.357
Prior probability for "buys_computer = yes": 0.643


In [4]:
def calculate_conditional_probs(feature, label, df):
    conditional_probs = {}
    classes = df[label].unique()

    for c in classes:
        class_data = df[df[label] == c]
        conditional_probs[c] = class_data[feature].value_counts(normalize=True).to_dict()

    return conditional_probs

for feature in df.columns[:-1]:
    conditional_probs = calculate_conditional_probs(feature, 'buys_computer', df)

    print(f"Class-conditional densities for {feature}:")
    for label, probs in conditional_probs.items():
        print(f"P({feature} | buys_computer = {label}):")
        for value, prob in probs.items():
            print(f"  P({feature} = {value} | buys_computer = {label}): {prob:.4f}")
        print()

Class-conditional densities for age:
P(age | buys_computer = no):
  P(age = <=30 | buys_computer = no): 0.6000
  P(age = >40 | buys_computer = no): 0.4000

P(age | buys_computer = yes):
  P(age = 31 . . .40 | buys_computer = yes): 0.4444
  P(age = >40 | buys_computer = yes): 0.3333
  P(age = <=30 | buys_computer = yes): 0.2222

Class-conditional densities for income:
P(income | buys_computer = no):
  P(income = high | buys_computer = no): 0.4000
  P(income = medium | buys_computer = no): 0.4000
  P(income = low | buys_computer = no): 0.2000

P(income | buys_computer = yes):
  P(income = medium | buys_computer = yes): 0.4444
  P(income = low | buys_computer = yes): 0.3333
  P(income = high | buys_computer = yes): 0.2222

Class-conditional densities for student:
P(student | buys_computer = no):
  P(student = no | buys_computer = no): 0.8000
  P(student = yes | buys_computer = no): 0.2000

P(student | buys_computer = yes):
  P(student = yes | buys_computer = yes): 0.6667
  P(student = no 

In [5]:
from scipy.stats import chi2_contingency

contingency_table = pd.crosstab(index=df['age'], columns=[df['income'], df['student'], df['credit rating']])
chi2, p, _, _ = chi2_contingency(contingency_table)

print(f"Chi-square statistic: {chi2}")
print(f"P-value: {p}")

alpha = 0.05
print(f"Significance level: {alpha}")
print(f"Is the p-value less than alpha? {'Yes' if p < alpha else 'No'}")

if p < alpha:
    print("Reject the null hypothesis. There is evidence of a significant association between the features.")
else:
    print("Fail to reject the null hypothesis. There is no significant evidence of an association between the features.")

Chi-square statistic: 12.95
P-value: 0.6764100579553458
Significance level: 0.05
Is the p-value less than alpha? No
Fail to reject the null hypothesis. There is no significant evidence of an association between the features.


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report


label_encoder = LabelEncoder()
for column in df.columns:
    df[column] = label_encoder.fit_transform(df[column])

X = df.drop('buys_computer', axis=1)
y = df['buys_computer']
Tr_X, Te_X, Tr_y, Te_y = train_test_split(X, y, test_size=0.2, random_state=42)

model = GaussianNB()
model.fit(Tr_X, Tr_y)

predictions = model.predict(Te_X)

accuracy = accuracy_score(Te_y, predictions)
print(f"Accuracy: {accuracy:.2f}")

print("Classification Report:")
print(classification_report(Te_y, predictions))

Accuracy: 0.67
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.67      1.00      0.80         2

    accuracy                           0.67         3
   macro avg       0.33      0.50      0.40         3
weighted avg       0.44      0.67      0.53         3



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics

df1 = pd.read_csv('./waterPotability_updated.csv')
print(df1.sample(20))

X = df1.drop('Potability', axis=1)
y = df1['Potability']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

nb_classifier = GaussianNB()
nb_classifier.fit(X_train, y_train)
y_pred = nb_classifier.predict(X_test)

accuracy = metrics.accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

print(metrics.classification_report(y_test, y_pred))

            ph    Hardness        Solids  Chloramines     Sulfate  \
1784  9.252857  168.040751  50279.262429     5.905056  415.450810   
540   3.841205  186.917765  19988.649262     7.174952  333.775777   
2743  8.277058  242.559246  12715.814280     5.128733  371.302747   
1189  6.859324  179.068805  27567.375670     5.885557  227.348460   
966   7.366915  185.043597  14151.760083     5.584369  324.570459   
1151  4.303575  227.007086   7323.302301     7.490508  326.695199   
1052  9.045721  222.184169  22829.110842     7.609833  293.062459   
1921  7.288737  233.977810  31617.075415     8.533888  331.838167   
3086  7.080795  184.614848  22289.727086     7.167405  392.462199   
3013  7.080795  215.431919  19909.938849     6.902628  299.086418   
3019  9.347345  201.632836  13749.596037     7.333742  322.044667   
1710  4.313219  191.119337  12770.831830     8.231574  284.945949   
1006  7.230845  177.574803  17864.689386     5.818229  343.593332   
1998  7.544306  211.051146  34359.

In [10]:
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier


X = df1.drop('Potability', axis=1)
y = df1['Potability']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print("Accuracy:", metrics.accuracy_score(y_test, y_pred))
print("Precision:", metrics.precision_score(y_test, y_pred))
print("Recall:", metrics.recall_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.602238046795524
Precision: 0.48633879781420764
Recall: 0.4671916010498688
              precision    recall  f1-score   support

           0       0.67      0.69      0.68       602
           1       0.49      0.47      0.48       381

    accuracy                           0.60       983
   macro avg       0.58      0.58      0.58       983
weighted avg       0.60      0.60      0.60       983



In [11]:
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print("Accuracy:", metrics.accuracy_score(y_test, y_pred))
print("Precision:", metrics.precision_score(y_test, y_pred))
print("Recall:", metrics.recall_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.6836215666327569
Precision: 0.6822916666666666
Recall: 0.3438320209973753
              precision    recall  f1-score   support

           0       0.68      0.90      0.78       602
           1       0.68      0.34      0.46       381

    accuracy                           0.68       983
   macro avg       0.68      0.62      0.62       983
weighted avg       0.68      0.68      0.65       983

